In [137]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [138]:
def load_pdf_data(file_path):
    # Creating a PyMuPDFLoader object with file_path
    loader = PyMuPDFLoader(file_path=file_path)
    
    # loading the PDF file
    docs = loader.load()
    
    # returning the loaded document
    return docs

In [139]:
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1024, chunk_overlap=20):
    
    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)
    
    # returning the document chunks
    return chunks

In [140]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )


# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # Saving the model in current directory
    vectorstore.save_local(storing_path)
    
    # returning the vectorstore
    return vectorstore

In [141]:
prompt = """
### System:
You are an AI Assistant that follows instructions extreamly well. \
Help as much as you can.
Please say I don't know if you don't know.

### User:
{prompt}

### Response:

"""

In [142]:
template = """
### System:
You are an respectful and honest assistant. You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
just say you don't know. Don't try to make up an answer.

### Context:
{context}

### User:
{question}

### Response:
"""

In [143]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )

In [144]:
def get_response(query, chain):
    # Getting response from chain
    response = chain({'query': query})
    
    # Wrapping the text for better output in Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    print(wrapped_text)

In [145]:
from langchain.llms import Ollama
from langchain import PromptTemplate

In [146]:
# Loading llama3:70b from Ollama
llm = Ollama(model="llama3:70b", temperature=0)

# Loading the Embedding Model
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

In [148]:
# loading and splitting the documents
docs = load_pdf_data(file_path="data/BERT_arxiv.pdf")
documents = split_docs(documents=docs)

# creating vectorstore
vectorstore = create_embeddings(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

In [149]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template(template)

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [150]:
get_response("Summarize the document", chain)

The document appears to be discussing the fine-tuning of BERT (Bidirectional Encoder Representations
from Transformers) on different natural language processing tasks. It provides illustrations of how
BERT can be adapted for various tasks, including single-sentence classification, sentence-pair
classification, and question-answering.  The document also mentions specific datasets and tasks,
such as SST-2 (Stanford Sentiment Treebank), CoLA (Corpus of Linguistic Acceptability), and STS-B.
Additionally, it cites several research papers related to BERT and its applications.  Furthermore,
the document discusses the results of fine-tuning BERT on a particular task, achieving improved
performance compared to baseline systems. It also mentions ablation studies conducted to understand
the importance of different facets of BERT.


In [151]:
get_response("What is QAOA", chain)

I don't know what QAOA is. The context provided doesn't mention QAOA. It mentions several other
acronyms such as MNLI, QQP, QNLI, SQuAD, and NSP, but not QAOA. If you can provide more information
or clarify what QAOA stands for, I may be able to help you better.


In [152]:
get_response("What is BERT", chain)

BERT (Bidirectional Encoder Representations from Transformers) is a deep learning model architecture
developed by Google that has gained popularity in natural language processing (NLP) tasks. It's a
multi-layer bidirectional transformer encoder based on the original implementation described in
Vaswani et al. (2017). BERT uses a unified architecture across different tasks and is pre-trained
using two unsupervised tasks: Masked Language Modeling (MLM) and Next Sentence Prediction (NSP).


In [ ]:
import os
pdf = os.listdir("./data")
for _ in pdf:
    print(_)

.DS_Store
A Tutorial on Quantum Approximate Optimization Algorithm QAOA Fundamentals and Applications.pdf
self_rag_arxiv.pdf
BERT_arxiv.pdf
crag_arxiv.pdf
RAG_arxiv.pdf
